# INFO-F-208

# Partie 1.1

Avant d'implémenter un algorithme qui calcule l’alignement entre deux séquences, vous implémentez deux ADT (Abstract Data Type):    


##  Un ADT séquence

Pour l'ADT _Sequence_ j'ai décidé de l'implémenter de façon à ce que son utilisation soit similaire à celle d'une _liste_ ou _string_. En effet ce qui nous interesse dedans c'est la séquence d'acide aminé qui est une suite de charactère et sera stocké en tant que _string_.

De plus une fonction _load_ statique à été ajouté à cette structure de donnée pour charger les séquences depuis un fichier _.fasta_.

In [2]:
class Sequence():
    """
    ADT séquence qui représente une séquence d’acides aminés
    et tous les opérations qu’on peut exécuter sur une séquence.
    """
    def __init__(self, sequence):
        self.sequence = sequence
    
    def __repr__(self):
        return self.sequence
    
    def __len__(self):
        return len(self.sequence)
    
    def __getitem__(self, index):
        """
        @desc: Permet d'interpreter la classe comme une "String".
        
        @param{index}: Index de la lettre qu'on veut consulter.
        """
        return self.sequence[index]

    @staticmethod
    def load(filename):
        """
        """
        with open(filename) as f:
            seq = ""
            for line in f:
                if line[0] == ">":
                    if seq:
                        yield Sequence(seq)
                    seq = ""
                else:
                    seq += line[:-1] # Not including '\n'
                    
            if (seq):
                yield Sequence(seq)

On charge l'ensemble des fichiers fournis dans l'énoncé.

In [3]:
sh3sequence = [seq for seq in Sequence.load("./SH3-sequence.fasta")]
maguk = [seq for seq in Sequence.load("./maguk-sequences.fasta")]

## Un ADT score qui représente une matrice de substitution et les opérations qu’on peut exécuter sur cette matrice.

À la manière de la structure de donnée _Sequence_ celle-ci imite la fonctionnement d'une matrice _normal_ c'est à dire qu'on peut récupérer les valeurs comme dans dans toute les matrices à la différence près qu'on va passer des acides aminés en paramètre pour récupérer leur _score_.

De plus la structure donnée inclue aussi une fonction _load_ pour charger les matrices de substitution qui nous ont été fournies dans l'énoncé.

In [4]:
class _ScoreLine:
    """
    ADT utilisée pour pouvoir utiliser l'ADT Score comme une matrice.
    """
    def __init__(self, line, indexes):
        self.line = line
        self.indexes = indexes
    
    def __getitem__(self, letter):
        j = self.indexes.index(letter)
        return self.line[j]

class Score:
    INDEXES = "ARNDCQEGHILKMFPSTWYV"
    def __init__(self, matrix = [[]], indexes = INDEXES):
        self.matrix = matrix
        self.indexes = indexes
        
    def __getitem__(self, letter): 
        """
        @desc: Renvoie le score d'une combinaison de deux acides
        
        @param{letter}: Le premier acide.
        @ex: Score['A']['Q']
        """
        i = self.indexes.index(letter)
        return _ScoreLine(self.matrix[i], self.indexes)

    def __repr__(self):
        ret = "  "
        for char in self.indexes:
            ret += " %3s " % (char)
        ret += '\n'
        i = 0
        for line in self.matrix:
            ret += "%3s " % (self.indexes[i])
            i += 1
            for char in line:
                if (char >= 0):
                    ret += ' '
                ret += "%3s " % (char)
            ret += '\n'

        return ret
    
    @staticmethod
    def load(filename):
        with open(filename) as f:
            matrix = []
            indexes = Score.INDEXES
            for line in f:
                if line[0] == ' ' or line[0] == '\t':
                    indexes = line.split()
                elif (line[0].isalpha()) or (line[0] == '*') or (line[0] == '-'):
                    l = line[1:]
                    matrix.append(list(map(int, l.split())))
                    
        return Score(matrix, indexes)

On charge les matrices

In [5]:
blosum80 = Score.load('./blosum80.txt')
blosum62 = Score.load('./blosum62.txt')
blosum40 = Score.load('./blosum40.txt')
pam120 = Score.load('./pam120.txt')
pam60 = Score.load('./pam60.txt')

# Partie 1.2 Implémentation de l'algorithme de Needleman-Wunsch

En utilisant les ADT construits pendant l'étape précédente, vous implémentez en Python l’algorithme Needleman‐Wunsch qui calcule l’alignement global en utilisant la pénalité affine.

In [6]:
# OPENING_GAP_PENALTY = -12
# EXTENDING_GAP_PENALTY = -2
OPENING_GAP_PENALTY = -4
EXTENDING_GAP_PENALTY = -1

In [7]:
def init_matrix(x, y):
    """
    Initialize the matrix with zeros and a column of base score.
    """
    result = [[0 for _ in range(y + 1)] for _ in range(x + 1)]
    for i in range(y + 1): result[0][i] = OPENING_GAP_PENALTY + i * EXTENDING_GAP_PENALTY
    for i in range(x + 1): result[i][0] = OPENING_GAP_PENALTY + i * EXTENDING_GAP_PENALTY
        
    return result

In [8]:
class NeedlemanWunsch:
    def __init__(self, seq1, seq2, scoringMatrix):
        self.seq1 = seq1
        self.seq2 = seq2
        self.scoringMatrix = scoringMatrix
        self.result = []
        
        self.m = len(seq1)
        self.n = len(seq2)
        
        self.S = init_matrix(self.m, self.n)
        self.V = init_matrix(self.m, self.n)
        self.W = init_matrix(self.m, self.n)
        
        for i in range(1, self.m + 1):
            for j in range(1, self.n + 1):
                # Voir slide 63 (L3 alignement de séquences)
                self.V[i][j] = max(
                    self.S[i - 1][j] + OPENING_GAP_PENALTY,
                    self.V[i - 1][j] + EXTENDING_GAP_PENALTY
                )

                self.W[i][j] = max(
                    self.S[i][j - 1] + OPENING_GAP_PENALTY,
                    self.W[i][j - 1] + EXTENDING_GAP_PENALTY
                )
            
                self.S[i][j] = max(
                    self.S[i - 1][j - 1] + scoringMatrix[self.seq1[i - 1]][self.seq2[j - 1]],
                    self.V[i][j],
                    self.W[i][j] 
                ) 
    
        self._NW("", "", self.m, self.n)
        
    def __repr__(self):
        ret = ""
        for i in range(0, len(self.result)):
            ret += "%s\n" % (self.result[i][0]) # seq1
            identity = 0
            similarity = 0
            gap = 0
            
            for j in range(0, len(self.result[i][0])):
                a = self.result[i][0][j]
                b = self.result[i][1][j]
                if (a == b):
                    ret += ':'
                    identity += 1
                    similarity += 1
                elif (a != '-' and b != '-' and self.scoringMatrix[a][b] >= 0):
                    ret += '.'
                    similarity += 1
                else:
                    ret += ' '
                    gap += 1
            ret += "\n"
            ret += "%s\n" % (self.result[i][1]) # seq2
            ret += "%3.3f%% identity\n" % (100 * identity / len(self.result[i][0]))
            ret += "%3.3f%% similarity\n" % (100 * similarity / len(self.result[i][0]))
            ret += "%3.3f%% gap\n"  % (100 * gap / len(self.result[i][0]))
            ret += "Length : %i\n" % (len(self.result[i][0]))
            ret += "Global score : %s\n" % (self.S[self.m][self.n])
        return ret

        
    def _NW(self, align1, align2, i, j):
        """
        """
        if (i > 0 and j > 0) and (self.S[i][j] == self.S[i - 1][j - 1] + self.scoringMatrix[self.seq1[i - 1]][self.seq2[j - 1]]):
            # diagonal
            self._NW(self.seq1[i - 1] + align1, self.seq2[j - 1] + align2, i - 1, j - 1)
        elif (i > 0) and (self.S[i][j] == self.V[i][j]):
            # top
            self._NW(self.seq1[i - 1] + align1, "-" + align2, i - 1, j)
        elif (j > 0) and (self.S[i][j] == self.W[i][j]):
            # left
            self._NW("-" + align1, self.seq2[j - 1] + align2, i, j - 1)
        else:
            # end of backtracking : we are back in S[0][0]
            self.result.append((align1, align2))
            

### Discussion

Comme vu lors du cours, l'alignement globale se fait comme suit

$$S(i,j) = max \
\begin{cases}  \
    S(i -1, j - 1) + t(i, j)\\ 
    [S(i - n_{gap1}, j) + g(n_{gap1})]_{1<ngap1<i} \\
    [S(i, j - n_{gap2}) + g(n_{gap2})]_{1<ngap2<j} \\
\end{cases}$$

Avec $t(i, j)$ qui est la valeur dans la matrice score passé en argument.

Cela peut être simplifier pour obtenir un temps d'exécution O(mn) en créant 3 matrices de la sorte:

$$V(i, j) = max \
\begin{cases}   \
    S(i-1, j) - O \\
    V(i-1, j) - E \\
\end{cases}$$

$$W(i, j) = max \
\begin{cases}   \
    S(i, j-1) - O \\
    W(i, j-1) - E \\
\end{cases}$$

$$S(i, j) = max \
\begin{cases}   \
    S(i-1, j-1) + t(i,j) \\
    V(i, j) \\
    W(i, j) \\
\end{cases}$$

Qui sont aussi présente dans mon impélementation. Ces matrices prennent aussi en compte les pénalitées liées à la création ou l'extension d'un _gap_. 

Pour créer l'alignement, il faut visualiser l'algorithme qui doit parcourir une matrice (la matrice S qu'on a créé jsute avant) en commançant en bas à droite pour arriver en haut à gauche.
L'algorithme doit trouver le chemin resolvant:

$$S(i, j) = \
\begin{cases}   \
    S(i-1, j-1) + t(i,j) \\
    V(i, j) \\
    W(i, j) \\
\end{cases}$$

Si la solution est:
 * $S(i, j) = S(i-1, j-1) + t(i,j)$ on a un déplacement diagonal traduit par deux AA égaux dans les deux séquences.
 * $S(i, j) = V(i, j)$ c'est un déplacement vers le haut traduit par un _gap_.
 * $S(i, j) = W(i, j)$ c'est équivalent à un déplacement vers la droite traduit par un _gap_.


#### La récursivité

Mon code utilise un algorithme récursif car de cette manière on peut si on le veut facilement le changer pour obtenir toutes les solutions optimales, ce que je n'ai pas fait ici car les solutions sont la plupart du temps triviales.

La méthode __NW__ doit être changée par: 

```
if i == 0 and j == 0:
    # end of backtracking : we are back in S[0][0]
    self.result.append((align1, align2))
    return
 
if (i > 0 and j > 0) and (self.S[i][j] == self.S[i - 1][j - 1] + self.scoringMatrix[self.seq1[i - 1]][self.seq2[j - 1]]):
    # diagonal
    self._NW(self.seq1[i - 1] + align1, self.seq2[j - 1] + align2, i - 1, j - 1)
if (i > 0) and (self.S[i][j] == self.V[i][j]):
    # top
    self._NW(self.seq1[i - 1] + align1, "-" + align2, i - 1, j)
if (j > 0) and (self.S[i][j] == self.W[i][j]):
    # left
    self._NW("-" + align1, self.seq2[j - 1] + align2, i, j - 1)
```

### Vérification

Vérifiez si votre programme donne le même résultat que l’outil LALIGN http://www.ch.embnet.org/software/LALIGN_form.html

Utilisez les séquences dans le fichier SH3-sequences.fasta sur le site web.        

In [9]:
print(NeedlemanWunsch(sh3sequence[0], sh3sequence[1], blosum80))

GGVTTFVALYDYESRTETD-LSFKKGERLQIVNNTEGD--WWLAHSLSTGQTGYIPSNYV--APSDS
  . . .: ::... :  : ::::.:. :... : : :  :. : .: .:. :.:: ::.   :   
--MEA-IAKYDFKA-TADDELSFKRGDILKVL-NEECDQNWYKA-EL-NGKDGFIPKNYIEMKP--H
35.821% identity
61.194% similarity
38.806% gap
Length : 67
Global score : 103



Le résultat de _LALIGN_ est le suivant:

-----------------------------------------------------------------------------------------

```
lobal/global (N-W) score: 114; 30.2% identity (61.9% similar) in 63 aa overlap (1-60:1-58)

               10        20        30         40        50          60         
unknow GGVTTFVALYDYESRTETDLSFKKGERLQIVNNT-EGDWWLAHSLSTGQTGYIPSNYV--APS
             .: ::... .. .::::.:. :...:   . .:. :.   .:. :.:: ::.   :      19
unknow ---MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAEL--NGKDGFIPKNYIEMKPH
                  10        20        30        40          50     
```

-----------------------------------------------------------------------------------------

On observe donc en utilisant la matrice BLOSUM80 des résultats très similaire à _LALIGN_ avec pour résultat un pourcentage de similarité très proche, 61.194% dans mon implémentation et 61.9% pour _LALIGN_. 
Quant à l'identité elle est de 35.8% dans mon implémentation et de 30.2% pour _LALIGN_, qui s'explique par un nombre plus élevé d'AA égaux entre les deux séquences dans mon implémentation.

In [10]:
print(NeedlemanWunsch(sh3sequence[0], sh3sequence[1], pam120))

GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTE-GD-WWLAHSLSTGQTGYIPSNYVA--PSDS
  . . .: ::. . .. .::::.:. :...:. : .. :  :  :. :. :.:: ::..  :   
--MEA-IAKYDFKATADDELSFKRGDILKVLNE-ECDQNWYKA-ELN-GKDGFIPKNYIEMKP--H
31.818% identity
63.636% similarity
36.364% gap
Length : 66
Global score : 92



### Résultat de _LALIGN_

-----------------------------------------------------------------------

```
 n-w opt:  57  Z-score: 157.8  bits: 32.6 E(1): 2.1e-27
global/global (N-W) score: 57; 29.0% identity (61.3% similar) in 62 aa overlap (1-62:1-58)

               10        20        30        40        50        60
unknow GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
         .   .: ::. . .. .::::.:. :...:.  .. :    :. :. :.:: ::..  
unknow MEA---IAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELN-GKDGFIPKNYIEMKPH
                  10        20        30        40         50               
```

-----------------------------------------------------------------------

Ici en utilisant la matrice de score PAM120 l'algorithme donne des résultats aussi similaire de la même manière qu'en utilisant la matrice BLOSUM80.

## Partie 1.3

Modifiez le logiciel de la partie 1.2 de sorte que on peut faire un alignement local (Smith‐Waterman). Utilisez les séquences dans le fichier maguk-sequences.fasta sur le site web. 

In [11]:
class SmithWaterman:
    def __init__(self, seq1, seq2, scoringMatrix, number = 1):
        self.seq1 = seq1
        self.seq2 = seq2
        self.scoringMatrix = scoringMatrix
        self.result = []
        
        self.m = len(seq1)
        self.n = len(seq2)
        
        self.S = init_matrix(self.m, self.n)
        self.V = init_matrix(self.m, self.n)
        self.W = init_matrix(self.m, self.n)
        
        # Remplit les matrices en utilisant les formules vue dans les slides du cour.
        for i in range(1, self.m + 1):
            for j in range(1, self.n + 1):
                # Voir slide 63 (L3 alignement de séquences)
                self.V[i][j] = max(
                    self.S[i - 1][j] + OPENING_GAP_PENALTY,
                    self.V[i - 1][j] + EXTENDING_GAP_PENALTY
                )

                self.W[i][j] = max(
                    self.S[i][j - 1] + OPENING_GAP_PENALTY,
                    self.W[i][j - 1] + EXTENDING_GAP_PENALTY
                )
            
                self.S[i][j] = max(
                    self.S[i - 1][j - 1] + scoringMatrix[self.seq1[i - 1]][self.seq2[j - 1]],
                    self.V[i][j],
                    self.W[i][j],
                ) 
    
        self.max = list()
        maxGen = self._find_maximum()
        for _ in range(number):
            x, y = next(maxGen)
            self.max.append([x, y])
            self._SW("", "", x, y)
        
    def __repr__(self):
        ret = ""
        for i in range(0, len(self.result)):
            ret += "%s\n" % (self.result[i][0]) # seq1
            identity = 0
            similarity = 0
            gap = 0
            for j in range(0, len(self.result[i][0])):
                a = self.result[i][0][j]
                b = self.result[i][1][j]
                if (a == b):
                    ret += ':'
                    identity += 1
                    similarity += 1
                elif (a != '-' and b != '-' and self.scoringMatrix[a][b] >= 0):
                    ret += '.'
                    similarity += 1
                else:
                    ret += ' '
                    gap += 1
            ret += "\n"
            ret += "%s\n" % (self.result[i][1]) # seq2
            ret += "%3.3f%% identity\n" % (100 * identity / len(self.result[i][0]))
            ret += "%3.3f%% similarity\n" % (100 * similarity / len(self.result[i][0]))
            ret += "%3.3f%% gap\n"  % (100 * gap / len(self.result[i][0]))
            ret += "Length : %i\n" % (len(self.result[i][0]))
            ret += "Global score : %s\n" % (self.S[self.max[i][0]][self.max[i][1]])
        return ret

    def _find_maximum(self):
        """
        Trouve le maximum dans la matrice
        """
        low = float("-inf")
        high = float("+inf")
        current_i, current_j = self.m, self.n
        while True:
            low = float("-inf")
            for i, line in enumerate(self.S):
                for j, value in enumerate(line):
                    if (value > low) and (value < high):
                        low = value
                        current_i, current_j = i, j
            high = low
            yield current_i, current_j
        
    def _SW(self, align1, align2, i, j):
        """
        """
        # print("---\nALIGN1: %s\nALIGN2: %s\n---" % (align1, align2))
        if (i > 0 and j > 0) and (self.S[i][j] > 0):
            if self.S[i][j] == self.S[i - 1][j - 1] + self.scoringMatrix[self.seq1[i - 1]][self.seq2[j - 1]]:
                # Vérification par apport à la diagonale.
                self._SW(self.seq1[i - 1] + align1, self.seq2[j - 1] + align2, i - 1, j - 1)
            elif self.S[i][j] == self.V[i][j]:
                # Vérification par apport à la gauche: trous dans seq2
                self._SW(self.seq1[i - 1] + align1, "-" + align2, i - 1, j)
            elif self.S[i][j] == self.W[i][j]:
                # Vérification par apport au dessus: trous sequ1
                self._SW("-" + align1, self.seq2[j - 1] + align2, i, j - 1)

        else:
            self.result.append((align1, align2))

### Discussion

L'algorithme est identique au précedent à l'exception qu'il ne commence pas son _chemin_ en bas à droite de la matrice mais commence à la plus grande valeur de disponible dans la matrice. La particularité est qu'on peut faire plusieurs alignements en cherchant les autres maximum dans la matrice (implémenté en donnant un 4ème arguemnt à la matrice)

Exemple de plusieurs alignements locaux

In [12]:
print(SmithWaterman(sh3sequence[0], sh3sequence[1], pam120, 5))

DYESRTETDLSFKKGERLQIVNNTE-GD-WWLAHSLSTGQTGYIPSNYVA--P
:. . .. .::::.:. :...:. : .. :  :  :. :. :.:: ::..  :
DFKATADDELSFKRGDILKVLNE-ECDQNWYKA-ELN-GKDGFIPKNYIEMKP
35.849% identity
69.811% similarity
30.189% gap
Length : 53
Global score : 99
DYESRTETDLSFKKGERLQIVNNTE-GD-WWLAHSLSTGQTGYIPSNYV
:. . .. .::::.:. :...:. : .. :  :  :. :. :.:: ::.
DFKATADDELSFKRGDILKVLNE-ECDQNWYKA-ELN-GKDGFIPKNYI
36.735% identity
71.429% similarity
28.571% gap
Length : 49
Global score : 98
DYESRTETDLSFKKGERLQIVNNTE-GD-WWLAHSLSTGQTGYIPSNYVA--PS
:. . .. .::::.:. :...:. : .. :  :  :. :. :.:: ::..  : 
DFKATADDELSFKRGDILKVLNE-ECDQNWYKA-ELN-GKDGFIPKNYIEMKPH
35.185% identity
68.519% similarity
31.481% gap
Length : 54
Global score : 97
DYESRTETDLSFKKGERLQIVNNTE-GD-WWLAHSLSTGQTGYIPSNY
:. . .. .::::.:. :...:. : .. :  :  :. :. :.:: ::
DFKATADDELSFKRGDILKVLNE-ECDQNWYKA-ELN-GKDGFIPKNY
37.500% identity
70.833% similarity
29.167% gap
Length : 48
Global score : 95
DYESRTETDLSFKKGERLQIVNNTE-GD-WWLAHSLSTGQTGYIPSNYV-
:. . .. .::::

### Vérification

In [13]:
import sys
sys.setrecursionlimit(10000)

print(SmithWaterman(maguk[0], maguk[1], pam120))

VPVIPVLPVPAENTVILPTIPQANPP--PVLVNTDSLETPT----Y--VNGTDADYEYEEITLERGNSGLGFSIAGGTDNPHIGDDSSIFITKIITGGAAAQDGRLRVNDCILRVNEVDVRDVTHSKAVEALKEAGSIVRLYVKRRKPVSEKIMEIKLIKGPKGLGFSIAGGVGNQHIPGDNSIYVTKIIEGGAAHKDGKLQIGDKLLAVNNVCLEEVTHEEAVTALKNTSDFVYLKVAKPTSMYMNDGYAPPDITNSSSQPVDNHVSP-SS--FLG--------QTPA--SPARYSPVSKAVLGDDEITREPRKVVLHRGSTGLGFNIVGGEDGEGIFISFILAGGPADLSGELRKGDRIISVNSVDLRAASHEQAAAALKNAGQAVTIVAQYRPEEYSRFEAKIHDLREQMMNSSISSGSGSLRTSQKRSLYVRALFDYDKTKDSGLPSQGLNFKFGDILHVINASDDEWWQARQVTPDGESDEVGVIPSKRRVEKKERARLKTVKFNSKT------RD-KGEIPDD-MGSKGLKHVTSNASDSESSYRGQEEYVLSYEPVNQQEVNYTRPVIILGPMKDRINDDLISEFPDKFGSCVPHTTRPKRDYEVDGRDYHFVTSREQMEKDIQEHKFIEAGQYNNHLYGTSVQSVREVAEKGKHCILDVSGNAIKRLQIAQLYPISIFIKPKSMENIMEMNKRLTEEQARKTFERAMKLEQEFTEHFTAIVQGDTLEDIYNQVKQIIEEQSGSYIWVPAKEKL
::  :  :::. .:   : .  ..:.  :     ..  : :    :  :::.:. . ::::.:::::::::::::::.::::. ::..::::::: ::::: :::: ::::.:::::::: .:.::.:::::::::..::: :.::.: .: :::..:.:::::::::::::.::::::::::::.:::::::::.:::.:::::.::::::. :..: :::::..:::::: :::::

### Résultat de LALIGN

Identique en utilisant PAM120.

```
 Waterman-Eggert score: 2711;  1234.7 bits; E(1) <  0
73.0% identity (90.2% similar) in 705 aa overlap (213-904:120-817)

            220       230       240       250       260       270  
unknow VNGTDADYEYEEITLERGNSGLGFSIAGGTDNPHIGDDSSIFITKIITGGAAAQDGRLRV
       :::.:. . ::::.:::::::::::::::.::::. ::..::::::: ::::: :::: :
unknow VNGSDGMFKYEEIVLERGNSGLGFSIAGGIDNPHVPDDPGIFITKIIPGGAAAMDGRLGV
     120       130       140       150       160       170         

            280       290       300       310       320       330  
unknow NDCILRVNEVDVRDVTHSKAVEALKEAGSIVRLYVKRRKPVSEKIMEIKLIKGPKGLGFS
       :::.:::::::: .:.::.:::::::::..::: :.::.: .: :::..:.:::::::::
unknow NDCVLRVNEVDVSEVVHSRAVEALKEAGPVVRLVVRRRQPPPETIMEVNLLKGPKGLGFS
     180       190       200       210       220       230         

            340       350       360       370       380       390  
unknow IAGGVGNQHIPGDNSIYVTKIIEGGAAHKDGKLQIGDKLLAVNNVCLEEVTHEEAVTALK
       ::::.::::::::::::.:::::::::.:::.:::::.::::::. :..: :::::..::
unknow IAGGIGNQHIPGDNSIYITKIIEGGAAQKDGRLQIGDRLLAVNNTNLQDVRHEEAVASLK
     240       250       260       270       280       290         

            400       410       420       430                      
unknow NTSDFVYLKVAKPTSMYMNDGYAPPDITNSSSQPVDNHVSPSSFLG-------------Q
       :::: :::::::: :. .:: ::::: ... .  .:::.: .: ::             .
unknow NTSDMVYLKVAKPGSLHLNDMYAPPDYASTFTALADNHISHNSSLGYLGAVESKVSYPAP
     300       310       320       330       340       350         

     440       450       460       470       480       490         
unknow TPASPARYSPVSKAVLGDDEITREPRKVVLHRGSTGLGFNIVGGEDGEGIFISFILAGGP
        ...:.::::... .:.....::::::..::.:::::::::::::::::::.::::::::
unknow PQVPPTRYSPIPRHMLAEEDFTREPRKIILHKGSTGLGFNIVGGEDGEGIFVSFILAGGP
     360       370       380       390       400       410         

     500       510       520       530       540       550         
unknow ADLSGELRKGDRIISVNSVDLRAASHEQAAAALKNAGQAVTIVAQYRPEEYSRFEAKIHD
       ::::::::.::::.:::.:.:: :.::::::::: :::.::::::::::::::::.::::
unknow ADLSGELRRGDRILSVNGVNLRNATHEQAAAALKRAGQSVTIVAQYRPEEYSRFESKIHD
     420       430       440       450       460       470         

     560       570       580       590       600       610         
unknow LREQMMNSSISSGSGSLRTSQKRSLYVRALFDYDKTKDSGLPSQGLNFKFGDILHVINAS
       :::::::::.::::::::::.:::::::::::::.:.:: ::::::.: .::::::::::
unknow LREQMMNSSMSSGSGSLRTSEKRSLYVRALFDYDRTRDSCLPSQGLSFSYGDILHVINAS
     480       490       500       510       520       530         

     620       630       640       650       660       670         
unknow DDEWWQARQVTPDGESDEVGVIPSKRRVEKKERARLKTVKFNSKTRDKGEIPDDMGSKGL
       :::::::: :::.:::...::::::.:::::::::::::::...:   : : .. .  ::
unknow DDEWWQARLVTPHGESEQIGVIPSKKRVEKKERARLKTVKFHART---GMIESNRDFPGL
     540       550       560       570       580          590      

     680       690       700       710       720       730         
unknow KHVTSNASDSESSYRGQEEYVLSYEPVNQQEVNYTRPVIILGPMKDRINDDLISEFPDKF
           :..  .. . .:::. .::::::..::..:.::::::::::::.:::::::::.::
unknow ----SDDYYGAKNLKGQEDAILSYEPVTRQEIHYARPVIILGPMKDRVNDDLISEFPHKF
            600       610       620       630       640       650  

     740       750       760       770       780       790         
unknow GSCVPHTTRPKRDYEVDGRDYHFVTSREQMEKDIQEHKFIEAGQYNNHLYGTSVQSVREV
       ::::::::::.:: ::::.:::::.::::::::::..:::::::.:..:::::.::::.:
unknow GSCVPHTTRPRRDNEVDGQDYHFVVSREQMEKDIQDNKFIEAGQFNDNLYGTSIQSVRAV
            660       670       680       690       700       710  

     800       810       820       830       840       850         
unknow AEKGKHCILDVSGNAIKRLQIAQLYPISIFIKPKSMENIMEMNKRLTEEQARKTFERAMK
       ::.::::::::::::::::: ::::::.:::::::.: .::::.: : ::: :....:::
unknow AERGKHCILDVSGNAIKRLQQAQLYPIAIFIKPKSIEALMEMNRRQTYEQANKIYDKAMK
            720       730       740       750       760       770  

     860       870       880       890       900    
unknow LEQEFTEHFTAIVQGDTLEDIYNQVKQIIEEQSGSYIWVPAKEKL
       ::::: : ::::::::.::.:::..:::::.::: :::::. :::
unknow LEQEFGEYFTAIVQGDSLEEIYNKIKQIIEDQSGHYIWVPSPEKL
            780       790       800       810       
```

Des scores très similaires à ceux de _LALIGN_ comme pour les alignements globaux à la différence que la séquence local choisis ne correspond pas exactement, surement dû à des différence de matrice de score différentes.

##   Retrouvez les similarités entre les 4 séquences. Expliquez les similarités.

Le résultat du site uniprot est le suivant: http://www.uniprot.org/align/A20161102F725F458AC8690F874DD868E4ED79B8896EBCFO .
Les similaritées sont dû à l'origine humaine des protéines.